In [10]:
import networkx as nx
import numpy as np
import copy
from numpy import linalg as LA
from numpy import random
n = 4039
e = 88234
average_deg = (2*e)/n
# Generating sparse graphs using networkx
G = nx.gnp_random_graph(n,average_deg/n)
L = nx.laplacian_matrix(G).toarray()
rng = np.random.default_rng()

# adjusting edge weights
for e in G.edges():
    G[e[0]][e[1]]['weight'] = 1


In [2]:
# For JL mechanism, to construct the edge adjacency matrix
def construct_edge_matrix(G):
    E = np.zeros((n*(n-1)//2, n))
    cnt = 0
    for e in G.edges():
        i = e[0]
        j = e[1]
        E[cnt][i] = np.sqrt(G[i][j]['weight'])
        E[cnt][j] = - np.sqrt(G[i][j]['weight'])
        cnt += 1
    return E


In [3]:
# subroutine of random walk
def weighted_sample_edges(edge_set):
    lit = np.zeros(n*(n-1)//2)
    space = {}
    current = 0
    for choice, weight in edge_set:
        if weight > 0:
            space[current] = choice
            current += weight
    rand = random.uniform(0, current)
    for key in sorted(space.keys() + [current]):
        if rand < key:
            return choice
        choice = space[key]
# sample from a set
def sample_set(s,m):
    s = list(s)
    choice = np.random.randint(0,m)
    return s[m]
# random walk procedure
def random_walk(G):
    E = set(G.edges())
    m = G.number_of_edge()
    t = m*np.log(n)
    lit =  lit = np.zeros(n*(n-1)//2)
    cnt = 0
    for e in G.edges():
        lit[cnt] = np.exp(G[e[0]][e[1]]['weight'])
    for i in range(0,t):
        e = sample_set(E,m)
        E.remove(e)
    choice = weighted_sample_edges(lit)
    E.add(choice)
    return E
    

In [15]:
# High pass filter
def alg(G):
    for e in G.edges():
         G[e[0]][e[1]]['weight'] += np.random.laplace(0, 100)
         if G[e[0]][e[1]]['weight'] < 10*np.log(n):
              G[e[0]][e[1]]['weight'] = 0

In [ ]:
# Spectral error
def spectral_error(L_inverse, u, v):
    n = L_inverse.shape[0]
    b = np.zeros((n,1))
    b[u] = 1
    b[v] = -1
    return np.dot(np.dot(b.T,L_inverse), b)[0,0]

def find_max_spectral_error(L):
    L_inverse = np.linalg.pinv(L)
    maxx = -1
    n = L_inverse.shape[0]
    for i in range(n):
        for j in range(n):
            if i != j:
                maxx = max(spectral_error(L_inverse, i, j), maxx)
    return maxx

In [4]:
# Naive Gaussian mechanism
def Gaussian(G):
    for i in range(0,n):
        for j in range(0,n):
            if i<j:
                noise = np.random.laplace(0, 1)
                if G.has_edge(i,j):
                    G[i][j]['weight']  += noise
                else:
                    G.add_edge(i, j, weight = noise)
    return G
        

In [5]:
# JL mechanism
def JL(G):
    r = 800*n
    w = 660*np.sqrt(n)
    for i in range(0,n):
        for j in range(0,n):
            if i<j:
                if G.has_edge(i,j):
                    G[i][j]['weight']  += w/n
                else:
                    G.add_edge(i, j, weight = w/n)
    E = construct_edge_matrix(G)
    s = np.random.normal(0, 1, size = (r, n*(n-1)//2))
    tmp = np.dot(s,E)
    return (1/r)*np.dot(tmp.T, tmp)
        

In [16]:
# Testing time
import timeit
G1 = copy.deepcopy(G)
start = timeit.default_timer()
alg(G1)
stop = timeit.default_timer()
print('Time of ours: ', stop - start)
print(G1.number_of_edges())


#G2 = copy.deepcopy(G)
#start = timeit.default_timer()
#Gaussian(G2)
#stop = timeit.default_timer()
#print('Time of Gaussian: ', stop - start)
#print(G2.number_of_edges())


# Tesing accuracy
L = nx.laplacian_matrix(G).toarray()
L1 = nx.laplacian_matrix(G1).toarray()
print('error of ours: ', LA.norm(L-L1,2))
#L2 = nx.laplacian_matrix(G2).toarray()
#print('error of Gaussian: ', LA.norm(L-L2,2))


Time of ours:  0.46400054200000795
87818
error of ours:  4754.367218662789


In [9]:
#test syntax
print(np.abs(np.random.laplace(0, 1)))

2.948819675788478
